In [14]:
%load_ext autoreload
%autoreload 2
#import glob2
import pandas as pd
import numpy as np
import text_summarization
import extract_text
import sentiment_analyzer
import keras
from tqdm import tqdm, tqdm_pandas
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import warnings
import csv
warnings.filterwarnings('ignore')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
#Load the df that we created in the other notebook
df_files = pd.read_csv("safe/df_file_final.csv", lineterminator='\n')

In [21]:
## Transform into array from string. Also rename column if a problem occured during loading.
from ast import literal_eval
if("sentences\r" in df_files.columns):
    df_files = df_files.rename(index=str,columns={'sentences\r': 'sentences'})
df_files.sentences = df_files.sentences.apply(literal_eval)

In [22]:
df_files.head(5).sentences[0]

([('Keppel Capital aims to create value and deliver sustainable returns for institutional and retail investors through a range of products including REITs, business trusts, private funds investing in real es tate in Asia, separate accounts and pooled investment vehicles.',
   5.163275694908415),
  ('Keppel Capital is a premier asset manager in Asia.', 10.307097586053025),
  ('Represent Keppel DC REIT in HSE commit tee meetings and annual safety events.',
   4.520748868940497),
  ('The asset managers under Keppel Capital include Keppel REIT Management Limited, Alpha Investment Partners Limited, Keppel Infrastructure Fund Management Pte Ltd and Keppel DC REIT Management P te Ltd.',
   20.267967689101564),
  ('It is the asset management arm of Keppel Corporation Limited (Keppel Corporation), a multinational conglomerate with key businesses in Offshore &',
   10.284593789539135),
  ('billion, Keppel Capital has a diversified portfolio that includes real estate, infrastructur e and data cen

In [23]:
##Method to predict the average sentiment of a list of phrase which are tuple containing (str,float) -> 
##(sentence,weight)
def average(tupple, sentimentAnalyzer):
    sents, imporances = zip(*tupple)
    sents = list(sents)
    importance = list(imporances)
    pred = np.squeeze(sentimentAnalyzer.predict(sents))
    return np.average(pred, weights=imporances)

##Return sentiment final given top_k phrases and company_phrases, with a hyper parameter to sum to a final result.
def find_sentiment(important_sents,sentimentAnalyzer):
    try:
        if important_sents == [-500]:
            return -500
        interest_sent, top_sent = important_sents
        lambda_ = 0.3
        return (1-lambda_)*average(interest_sent,sentimentAnalyzer) + \
            lambda_*average(top_sent, sentimentAnalyzer)
    except:
        return -999

In [30]:
#model = keras.models.load_model('models/my_amazon_model.h5')
analyzser = sentiment_analyzer.SentimentAnalyzer('models/my_amazon_model.h5')
tqdm().pandas(desc="Progress:")
df_files.sentiment = df_files.progress_apply(lambda row: np.mean(find_sentiment(row['sentences'],analyzser)),axis=1)

10000










0it [00:00, ?it/s]















Progress::   0%|          | 0/387 [00:00<?, ?it/s]








  0%|          | 0/6 [00:00<?, ?it/s]








100%|██████████| 6/6 [00:00<00:00, 188.18it/s]








  0%|          | 0/5 [00:00<?, ?it/s]








100%|██████████| 5/5 [00:00<00:00, 268.46it/s]







Progress::   1%|          | 2/387 [00:00<01:00,  6.38it/s]








  0%|          | 0/38 [00:00<?, ?it/s]








 68%|██████▊   | 26/38 [00:00<00:00, 254.85it/s]








100%|██████████| 38/38 [00:00<00:00, 251.96it/s]

(6,)
(5,)











  0%|          | 0/5 [00:00<?, ?it/s]








100%|██████████| 5/5 [00:00<00:00, 96.96it/s]







Progress::   1%|          | 3/387 [00:00<01:20,  4.77it/s]








  0%|          | 0/17 [00:00<?, ?it/s]








100%|██████████| 17/17 [00:00<00:00, 202.91it/s]

(38,)
(5,)


...

(259,)
(5,)











 50%|█████     | 23/46 [00:00<00:00, 105.39it/s]








 76%|███████▌  | 35/46 [00:00<00:00, 106.22it/s]








100%|██████████| 46/46 [00:00<00:00, 115.65it/s]








  0%|          | 0/5 [00:00<?, ?it/s]








100%|██████████| 5/5 [00:00<00:00, 52.33it/s]







Progress::  97%|█████████▋| 376/387 [02:21<00:01,  6.49it/s]








  0%|          | 0/113 [00:00<?, ?it/s]

(46,)
(5,)











 13%|█▎        | 15/113 [00:00<00:00, 133.48it/s]








 26%|██▌       | 29/113 [00:00<00:00, 131.09it/s]








 36%|███▋      | 41/113 [00:00<00:00, 126.08it/s]








 47%|████▋     | 53/113 [00:00<00:00, 122.15it/s]








 58%|█████▊    | 66/113 [00:00<00:00, 121.34it/s]








 72%|███████▏  | 81/113 [00:00<00:00, 125.57it/s]








 90%|█████████ | 102/113 [00:00<00:00, 135.23it/s]








100%|██████████| 113/113 [00:00<00:00, 139.98it/s]








  0%|          | 0/5 [00:00<?, ?it/s]








100%|██████████| 5/5 [00:00<00:00, 211.11it/s]







Progress::  99%|█████████▉| 385/387 [02:22<00:00,  6.93it/s]







Progress:: 100%|██████████| 387/387 [02:22<00:00,  2.71it/s]

(113,)
(5,)


In [ ]:
##Get the index of the top and worst document from a dataframe with value filled.
"""Return : 4 array of index that correspond to worst and best document of each company"""
def getSubmission(df):
    dfCleaned = df[df['sentiment']>0.01]
    top_keppel = list(dfCleaned[dfCleaned['company'] == 'Keppel'].sort_values('sentiment',ascending=False).head(10).id)
    down_keppel = list(dfCleaned[dfCleaned['company'] == 'Keppel'].sort_values('sentiment',ascending=True).head(10).id)
    top_prud = list(dfCleaned[dfCleaned['company'] == 'Prudential'].sort_values('sentiment',ascending=False).head(10).id)
    down_prud = list(dfCleaned[dfCleaned['company'] == 'Prudential'].sort_values('sentiment',ascending=True).head(10).id)
    return top_keppel,down_keppel, top_prud, down_prud

In [ ]:
"""Create a CSV submission from the index of top/worst document of each company"""
def createSubCSV(topId_k,downId_k,topId_p,downId_p):
    with open("sub.csv",'w',newline='') as csvfile:
        spamwritter = csv.writer(csvfile)
        spamwritter.writerow(['id','rank'])
        for i in range(len(df_files.sentiment)):
            if(i in topId_k):
                spamwritter.writerow([i,(10-topId_k.index(i))])
            elif(i in downId_k):
                spamwritter.writerow([i,(downId_k.index(i)-10)])
            elif(i in topId_p):
                spamwritter.writerow([i,(10-topId_p.index(i))])
            elif(i in downId_p):
                spamwritter.writerow([i,(downId_p.index(i)-10)])
            else:
                spamwritter.writerow([i,0])

In [ ]:
top_k,down_k, top_p, down_p = [],[],[],[]
createSubCSV(top_k,down_k, top_p, down_p)